<a href="https://colab.research.google.com/github/mabonmn/Clip/blob/main/Clip_Features_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-n4_mwelx
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-n4_mwelx
     |████████████████████████████████| 53 kB 415 kB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369328 sha256=31e3fecabb390fb5abf6a01f20a7d1e6564ff889767f13e965936793ee8179bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-1l5a97z4/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
import torch
import clip
from PIL import Image
import numpy as np
import os
import cv2
import time
import h5py
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#constants
video_path="/content/drive/MyDrive/Feature Extraction - Mabon/Clip_Features- P1/"              #video address
frame_path="/content/"                                                                         #video frames address
timeF=15                                                                                       #Video frame count interval frequency

H5file_old="/content/drive/MyDrive/Feature Extraction - Mabon/Clip_Features- P1/eccv16_dataset_summe_google_pool5_old.h5"

video_num=1                                                                                     #enter the num to process videos you want,use 1 to check
#we have formated the vidoes and loaded it correctly. 
#need to work on H5file. 

In [ ]:
#Create a temporary storage frame
for i in range(video_num):
    isExists = os.path.exists(frame_path+str(i+1))
    
    if not isExists:
        os.makedirs(frame_path+str(i+1))
        print(" The directory %s was created successfully"%(i+1))
    else:

        continue


 The directory 1 was created successfully


In [ ]:
for i in range(video_num):
    vc = cv2.VideoCapture(video_path + str(i+1)+'.mp4')
    if vc.isOpened(): #whether to open normally
        rval , frame = vc.read()
    else:
        rval = False
    print('video_{0} open {1}'.format(i+1,rval)) 
    # Read video frames in loop
    c=0#start
    while rval:  
        rval, frame = vc.read()
        #print("Fram Number; "+str(int(c / timeF)))
        if (c % timeF == 0):  # Store operation every timeF frame
            frames=frame_path+"%s/"%(i+1)
            cv2.imwrite("%s/"%frames+ str(int(c / timeF)) + '.png', frame)
        
        c = c + 1
# what is causing this error? only single frame is loaded into the folder after vid no 15. 
#( TVSUM DATA SET ERROR) Hardware error
#vid 20
#vid 28
#vid 50

video_1 open True


In [ ]:
def filecout(path):
    all_files = os.listdir(path)
    # COUNT
    file_num = 0
    for j in all_files:
        file_num += 1
    return file_num

In [ ]:
if torch.cuda.is_available():
    device = "cuda" 
else:
    device = "cpu" 
    print("cpu")
model, preprocess = clip.load("RN50", device=device)

cpu


100%|████████████████████████████████████████| 244M/244M [00:02<00:00, 103MiB/s]


In [ ]:
#check the old features shape with new features
shape = []
for j in range(video_num):
    f2 = h5py.File(H5file_old, 'r')     # open the file
    data = f2["video_%s/features"%(j+1)]
    
    frames=frame_path+"%s/"%(j+1)
    frames_num=filecout("%s/" %frames)
    shape.append(data.shape[0])
    if frames_num != data.shape[0]:
        print("%s th folder need to correct "%(j+1),end="")
        print(data.shape[0])

print("correct shape")
print(shape)
f2.close()
#copy the last picture to run it 

correct shape
[300]


In [ ]:
start=time.time()
for j in range(video_num): 
    features = []
    frames=frame_path+"%s/"%(j+1)
    frames_num=filecout("%s/" %frames)
    for i in range(frames_num): #preprocess image to clip Format
        image = preprocess(Image.open(frames+str(i)+'.png')).unsqueeze(0).to(device)
        with torch.no_grad():#extract the features from encoder 
            a=model.encode_image(image).detach().cpu().numpy()
            features.append(a)
          
    features = np.concatenate(features, axis=0)
endtime=time.time()-start
print("time：{} s".format(endtime))

time：139.56616473197937 s


In [ ]:
features

array([[-0.03819315, -0.02809658, -0.01071962, ...,  0.01487098,
         0.0146119 , -0.01346372],
       [-0.04658166, -0.02260109, -0.01168687, ...,  0.02008178,
         0.01135532, -0.0102795 ],
       [-0.04629778, -0.01651353, -0.00819843, ...,  0.01618402,
         0.00958537, -0.00068036],
       ...,
       [-0.03901787, -0.01977705, -0.01006909, ...,  0.01298362,
         0.01087531, -0.00507459],
       [-0.03665117, -0.01902382, -0.00727616, ...,  0.01977888,
         0.00616428, -0.00243039],
       [-0.0448706 , -0.01539124, -0.01065195, ...,  0.02303245,
         0.00900167, -0.00528508]], dtype=float32)

#Sample image

In [ ]:
print(clip.available_models())

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']


In [ ]:
image = preprocess(Image.open('test.jpg')).unsqueeze(0).to(device)
feature_sample=model.encode_image(image).detach().cpu().numpy()

FileNotFoundError: ignored

In [ ]:
feature_sample.shape